In [ ]:
import os
from pathlib import Path
import cv2
# 填入data_mp4文件夹的存储路劲
# source_base_path = "F:\\bic\\"
source_base_path = Path(__file__).resolve.parent.parent.parent / "bic"
source_path = os.path.join(source_base_path, "data\\data_mp4")
save_base_path = "F:\\dachuang_network\\"
save_path = os.path.join(save_base_path, "data\\raw\\")
name_dict = {"福田东": "cam1", "福田北": "cam2", "福田南": "cam3", "福田西": "cam4"}
if not os.path.exists(save_path):
            # 如果文件夹不存在，则创建新的文件夹
            os.makedirs(save_path)
            print("{}文件夹已创建".format(save_path))
print(source_path)
print(save_path)

In [ ]:
def get_dirs_list(source_path):
    dirs_name_list = ['福田东', '福田北', '福田南', '福田西']
    dirs_list = os.listdir(source_path)
    for name in dirs_list.copy():
        print(name)
        if name not in dirs_name_list:
            dirs_list.remove(name)
        
    return [os.path.join(source_path, name) for name in dirs_list]
list = get_dirs_list(source_path)
print(list)

In [ ]:
def extract_frame_per_name_fast(file_path):
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    normalized = file_name.replace(" ", "_")
    parts = normalized.split('_')
    date = parts[0]          # 日期（如 2025-03-06）
    hour = parts[1]          # 小时（如 14）
    location = name_dict[parts[-1]]     # 地点（如 福田东，始终在最后）
    
    return f"{location}_{date}_{hour}"

In [ ]:
def get_frames(dirs_list, save_path):
    # 1. 创建日间/夜间目录

    # 2. 遍历所有视频目录
    for video_dir in dirs_list:
        day_dir = os.path.join(save_path, name_dict[video_dir.split("\\")[-1]], "day")
        night_dir = os.path.join(save_path, name_dict[video_dir.split("\\")[-1]], "night")
        os.makedirs(day_dir, exist_ok=True)
        os.makedirs(night_dir, exist_ok=True)
        
        # 3. 处理目录中的每个视频文件
        for video_file in os.listdir(video_dir):
            video_path = os.path.join(video_dir, video_file)
        
            # 跳过非视频文件
            if not video_file.lower().endswith(('.mp4', '.avi', '.mov')):
                continue

            # 4. 提取关键信息 (地点_日期_小时)
            try:
                frame_per_name = extract_frame_per_name_fast(video_path)
                _, _, hour_str = frame_per_name.split('_')  # 拆分出小时
                hour = int(hour_str)
            except Exception as e:
                print(f"Error parsing {video_file}: {e}")
                continue

            # 5. 判断日间/夜间
            time_dir = day_dir if 6 <= hour <= 20 else night_dir
            time_label = "day" if 6 <= hour <= 20 else "night"

            # 6. 抽帧处理
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Failed to open {video_file}")
                continue

            frame_count = 0
            frame_interval = 4  # 每4帧保存1帧
            saved_frame_count = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                if frame_count % frame_interval == 0:
                    # 生成标准化文件名 (如 cam1_2025-03-06_14_frame_001.jpg)
                    frame_name = f"{frame_per_name}_frame_{saved_frame_count+1:03d}.jpg"
                    cv2.imwrite(os.path.join(time_dir, frame_name), frame)
                    saved_frame_count += 1
                
                frame_count += 1

            cap.release()
            print(f"Processed {video_file} ({time_label}): saved {saved_frame_count} frames to {time_dir}")
get_frames(get_dirs_list(source_path), save_path)